<a href="https://colab.research.google.com/github/jzheng23/colab/blob/main/Smartphone_screeentime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# smart phone screentime

- timestamp
- survey
- screenshot ??

# Firebase database

## Import from Firebase and save as csv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import firebase_admin
from firebase_admin import credentials, db
import csv
import pandas as pd

In [6]:
#Jian
google_drive_path = '/content/drive/MyDrive/Problematic Smartphone Usage/Ambient display/data'

In [ ]:
#Subin
google_drive_path = '/content/drive/MyDrive/UMD_research/Problematic_Smartphone_Usage'

In [4]:
cred = credentials.Certificate(google_drive_path+'/timer-42ad2-firebase-adminsdk-4r7oj-2c373565f2.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://timer-42ad2-default-rtdb.firebaseio.com'
})

In [26]:
#Convert Unix timestamp to date/datetime in specified timezone
def convert_unix_to_date(unix_timestamp, timezone='America/New_York', return_date=True):
   """

   Parameters:
       unix_timestamp: Unix timestamp in milliseconds
       timezone: String of timezone (default 'America/New_York')
       return_date: If True returns date only, if False returns datetime
   """
   dt = pd.to_datetime(unix_timestamp, unit='ms', utc=True).tz_convert(timezone)
   return dt.date() if return_date else dt

In [42]:
# this function works for device and timezone infor
def firebase_to_csv(ref_path, output_file):
    # Get reference to device_info
    ref = db.reference(ref_path)
    data = ref.get()

    transformed_data = []

    # Transform the nested structure
    for pid, pid_data in data.items():
        pid = pid.lower()

        if isinstance(pid_data, str):
            # Case where timezone is directly under pid
            row = {
                'pid': pid,
                'host': pd.NA,  # or None
                'timezone': pid_data
            }
            transformed_data.append(row)
        else:
            # Case where pid has host-timezone pairs
            for host, timezone in pid_data.items():
                row = {
                    'pid': pid,
                    'host': host,
                    'timezone': timezone
                }
                transformed_data.append(row)

    # Convert to DataFrame
    df = pd.DataFrame(transformed_data)

    # Ensure pid and host are the first columns
    cols = ['pid', 'host'] + [col for col in df.columns if col not in ['pid', 'host']]
    df = df[cols]

    # Save to CSV
    df.to_csv(output_file, index=False)

### Device info

In [6]:
firebase_to_csv('/device_info','device_info.csv')
device_data0 = pd.read_csv('/content/device_info.csv')
device_data0

,pid,host,timezone
0,pi005,r-8891092928c26a74-fc4w,"{'Base': '15', 'Brand': 'google', 'Model': 'Pi..."
1,pi010,660225,"{'Base': '15', 'Brand': 'google', 'Model': 'Pi..."
2,pi010,r-8891092928c26a74-fc4w,"{'Base': '15', 'Brand': 'google', 'Model': 'Pi..."
3,pi011,r-8891092928c26a74-fc4w,"{'Base': '15', 'Brand': 'google', 'Model': 'Pi..."
4,te006,r-8891092928c26a74-v0fg,"{'Base': '13', 'Brand': 'google', 'Model': 'sd..."
5,te010,r-8891092928c26a74-fc4w,"{'Base': '15', 'Brand': 'google', 'Model': 'Pi..."


In [7]:
# Convert the timestamps
device_data = device_data0.copy()
# Convert the timezone strings to dictionaries
device_data['timezone'] = device_data['timezone'].apply(eval)

# Create new columns from the dictionary values
device_data['Base'] = device_data['timezone'].apply(lambda x: x['Base'])
device_data['Brand'] = device_data['timezone'].apply(lambda x: x['Brand'])
device_data['Model'] = device_data['timezone'].apply(lambda x: x['Model'])

# Drop the original timezone column
device_data = device_data.drop('timezone', axis=1)
device_data

In [9]:
# Save to google drive
device_data.to_csv(google_drive_path + '/device_info.csv', index=False)

### Timezone info

In [10]:
firebase_to_csv('/timezones','timezones.csv')
timezones0 = pd.read_csv('/content/timezones.csv')
timezones0

,pid,host,timezone
0,pi001,NaN,America/New_York
1,pi002,NaN,America/New_York
2,pi003,NaN,America/New_York
3,pi004,NaN,America/New_York
4,pi005,r-8891092928c26a74-fc4w,America/New_York
5,pi010,660225,America/New_York
6,pi010,r-8891092928c26a74-fc4w,America/New_York
7,pi011,r-8891092928c26a74-fc4w,America/New_York
8,te001,NaN,America/New_York
9,te003,NaN,America/New_York


In [11]:
timezone_data = timezones0.copy()
timezone_data.to_csv(google_drive_path + '/timezones.csv', index=False)

### Response data

In [12]:
def process_responses(responses, pid, host, survey_id):
    responses_data = []
    response_dict = {
        'pid': pid,
        'host': host,
        'surveyID': survey_id
    }

    if isinstance(responses, dict):
        for question_id, answer in responses.items():
            response_dict[f'q_{question_id}'] = answer
    elif isinstance(responses, list):
        for question_id, answer in enumerate(responses):
            if answer is not None:
                response_dict[f'q_{question_id}'] = answer

    responses_data.append(response_dict)
    return responses_data

def get_timestamp(data):
    if isinstance(data, dict):
        return data.get('a') or data.get('timestamp')
    return None

def get_responses(data):
    if isinstance(data, dict):
        return data.get('b') or data.get('responses')
    elif isinstance(data, list):
        return data
    return None

def is_host_level(data):
    """Check if this dictionary represents a host level"""
    if isinstance(data, dict):
        # Check if any value contains 'responses' or 'timestamp' keys
        for value in data.values():
            if isinstance(value, dict):
                if 'responses' in value or 'timestamp' in value:
                    return True
    return False

In [13]:
def responses_to_csvs(meta_file, responses_file):
    ref = db.reference('/responses')
    data = ref.get()

    meta_data = []
    responses_data = []

    for pid, pid_data in data.items():
        pid = pid.lower()
        if not pid_data:
            continue

        if isinstance(pid_data, list):
            for survey_id, survey_data in enumerate(pid_data):
                if survey_data:
                    timestamp = get_timestamp(survey_data)

                    meta_data.append({
                        'pid': pid,
                        'host': pd.NA,
                        'surveyID': str(survey_id),
                        'timestamp': timestamp
                    })

                    responses = get_responses(survey_data)
                    if responses:
                        responses_data.extend(process_responses(responses, pid, pd.NA, str(survey_id)))
        else:
            # Check each item under pid
            for key1, value1 in pid_data.items():
                if isinstance(value1, dict):
                    if is_host_level(value1):
                        # This is a host
                        host = key1
                        for survey_id, survey_data in value1.items():
                            if isinstance(survey_data, dict):
                                timestamp = get_timestamp(survey_data)

                                meta_data.append({
                                    'pid': pid,
                                    'host': host,
                                    'surveyID': survey_id,
                                    'timestamp': timestamp
                                })

                                responses = get_responses(survey_data)
                                if responses:
                                    responses_data.extend(process_responses(responses, pid, host, survey_id))
                    else:
                        # This is a survey (no host)
                        survey_id = key1
                        survey_data = value1
                        timestamp = get_timestamp(survey_data)

                        meta_data.append({
                            'pid': pid,
                            'host': pd.NA,
                            'surveyID': survey_id,
                            'timestamp': timestamp
                        })

                        responses = get_responses(survey_data)
                        if responses:
                            responses_data.extend(process_responses(responses, pid, pd.NA, survey_id))

    # Create and save metadata DataFrame
    meta_df = pd.DataFrame(meta_data)
    meta_df = meta_df[['pid', 'host', 'surveyID', 'timestamp']]  # ensure column order
    meta_df.to_csv(meta_file, index=False)

    # Create responses DataFrame in wide format
    responses_df = pd.DataFrame(responses_data)

    # Ensure the first columns are in the correct order
    first_cols = ['pid', 'host', 'surveyID']

    # Get question columns and sort them numerically
    q_cols = [col for col in responses_df.columns if col.startswith('q_')]
    q_cols.sort(key=lambda x: int(x.split('_')[1]))  # Sort by the number after 'q_'

    # Combine columns in correct order
    responses_df = responses_df[first_cols + q_cols]

    responses_df.to_csv(responses_file, index=False)

In [14]:
# Example usage
responses_to_csvs('survey_meta.csv', 'survey_responses.csv')

#### Meta data

In [ ]:
survey_meta0 = pd.read_csv('/content/survey_meta.csv')
survey_meta0

In [25]:
survey_data = survey_meta0.copy()
survey_data['submitDate'] = survey_data['timestamp'].apply(lambda x: convert_unix_to_date(x))
survey_data

,pid,host,surveyID,timestamp,submitDate
0,pi001,NaN,1,1731445507251,2024-11-12
1,pi001,NaN,2,1731689878360,2024-11-15
2,pi002,NaN,1,1731555141456,2024-11-13
3,pi002,NaN,2,1731715678449,2024-11-15
4,pi003,NaN,1,1731452602347,2024-11-12
5,pi003,NaN,2,1731722935616,2024-11-15
6,pi003,NaN,3,1731896518856,2024-11-17
7,pi004,NaN,1,1731586828153,2024-11-14
8,pi004,NaN,2,1731848139596,2024-11-17
9,pi004,NaN,3,1732102200238,2024-11-20


In [23]:
# Pivot the dataframe to create survey columns
survey_meta_pivoted = survey_data.pivot(index=['pid','host'], columns='surveyID', values='submitDate')

# Rename the columns
survey_meta_pivoted.columns = ['survey' + str(col) for col in survey_meta_pivoted.columns]

# Reset the index to make pid a column again
survey_meta_pivoted = survey_meta_pivoted.reset_index()


,pid,host,survey1,survey2,survey3,survey4
0,pi001,NaN,2024-11-12,2024-11-15,NaN,NaN
1,pi002,NaN,2024-11-13,2024-11-15,NaN,NaN
2,pi003,NaN,2024-11-12,2024-11-15,2024-11-17,NaN
3,pi004,NaN,2024-11-14,2024-11-17,2024-11-20,NaN
4,pi005,NaN,2024-11-15,NaN,NaN,NaN
5,te001,NaN,2024-11-12,2024-11-12,2024-11-12,2024-11-12
6,te003,NaN,2024-11-11,2024-11-11,2024-11-11,NaN


In [32]:
# Save to google drive
survey_meta_pivoted.to_csv(google_drive_path + '/survey_meta.csv', index=False)

#### Response data

In [16]:
survey_responses0 = pd.read_csv('/content/survey_responses.csv')
survey_responses0

,pid,host,surveyID,q_0,q_1,q_2,q_3,q_4,q_5,q_6,...,q_18,q_19,q_20,q_21,q_22,q_23,q_24,q_25,q_26,q_27
0,pi001,NaN,1,3,3,0,0,0,2,3,...,3.0,3.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN
1,pi001,NaN,2,0,2,3,2,2,1,3,...,2.0,3.0,4.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN
2,pi002,NaN,1,3,2,3,1,4,1,0,...,2.0,4.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
3,pi002,NaN,2,0,1,1,2,4,0,0,...,2.0,2.0,3.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN
4,pi003,NaN,1,3,3,3,2,1,3,3,...,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN
5,pi003,NaN,2,3,3,3,3,1,3,2,...,1.0,3.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
6,pi003,NaN,3,3,1,1,2,1,2,2,...,2.0,3.0,3.0,2.0,1.0,3.0,3.0,2.0,2.0,1.0
7,pi004,NaN,1,4,3,3,1,0,3,3,...,2.0,3.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
8,pi004,NaN,2,1,2,2,2,0,3,3,...,1.0,4.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN
9,pi004,NaN,3,4,2,2,2,0,2,3,...,0.0,4.0,2.0,2.0,1.0,4.0,3.0,1.0,1.0,1.0


*TODO* still cannot read the host

In the real data there should always be a host

This should work for the real data, not tested yet

In [ ]:
def process_responses(responses, pid, host, survey_id):
    responses_data = []
    response_dict = {
        'pid': pid,
        'host': host,
        'surveyID': survey_id
    }

    if isinstance(responses, dict):
        for question_id, answer in responses.items():
            response_dict[f'q_{question_id}'] = answer
    elif isinstance(responses, list):
        for question_id, answer in enumerate(responses):
            if answer is not None:
                response_dict[f'q_{question_id}'] = answer

    responses_data.append(response_dict)
    return responses_data

def get_timestamp(data):
    if isinstance(data, dict):
        return data.get('a') or data.get('timestamp')
    return None

def get_responses(data):
    if isinstance(data, dict):
        return data.get('b') or data.get('responses')
    elif isinstance(data, list):
        return data
    return None

def responses_to_csvs(meta_file, responses_file):
    ref = db.reference('/responses')
    data = ref.get()

    meta_data = []
    responses_data = []

    for pid, pid_data in data.items():
        if not pid_data:
            continue

        # Process each host under pid
        for host, host_data in pid_data.items():
            # Process each survey under host
            for survey_id, survey_data in host_data.items():
                timestamp = get_timestamp(survey_data)

                meta_data.append({
                    'pid': pid,
                    'host': host,
                    'surveyID': survey_id,
                    'timestamp': timestamp
                })

                responses = get_responses(survey_data)
                if responses:
                    responses_data.extend(process_responses(responses, pid, host, survey_id))

    # Create and save metadata DataFrame
    meta_df = pd.DataFrame(meta_data)
    meta_df = meta_df[['pid', 'host', 'surveyID', 'timestamp']]  # ensure column order
    meta_df.to_csv(meta_file, index=False)

    # Create responses DataFrame in wide format
    responses_df = pd.DataFrame(responses_data)

    # Ensure the first columns are in the correct order
    first_cols = ['pid', 'host', 'surveyID']

    # Get question columns and sort them numerically
    q_cols = [col for col in responses_df.columns if col.startswith('q_')]
    q_cols.sort(key=lambda x: int(x.split('_')[1]))

    # Combine columns in correct order
    responses_df = responses_df[first_cols + q_cols]

    responses_df.to_csv(responses_file, index=False)

### Ringer

#### Counters

In [ ]:
def firebase_to_csv_counters(ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []
    for pid, value in data.items():
        pid = pid.lower()
        # Case 1: only numeric
        if isinstance(value, int):
            transformed_data.append({'pid': pid, 'screen_event_count': value})

        # Case 2: dictionary => selecting only value
        elif isinstance(value, dict):
            numeric_values = [v for v in value.values() if isinstance(v, int)]
            if numeric_values:
                total = sum(numeric_values)
                transformed_data.append({'pid': pid, 'screen_event_count': total})

    df = pd.DataFrame(transformed_data)
    df = df[['pid', 'screen_event_count']]
    df.to_csv(output_file, index=False)

In [ ]:
firebase_to_csv_counters('/ringer_event_counters','ringer_event_counters.csv')
ringer_event_counters_data0 = pd.read_csv('/content/ringer_event_counters.csv')
ringer_event_counters_data0

,pid,screen_event_count
0,pi005,5
1,pi010,12


#### Mode Events

In [ ]:
def firebase_to_csv_log_mode(ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    for pid, middle_levels in data.items():
        pid = pid.lower()
        for middle_key, sub_changes in middle_levels.items():
            # process: checking the structure of sub_data (dic or list)
            if isinstance(sub_changes, dict):
                for sub_key, event in sub_changes.items():
                    if isinstance(event, dict):
                        row = {'pid': pid, 'label': sub_key,
                            'mode': event.get('mode'),
                            'timestamp': event.get('timestamp')}
                        transformed_data.append(row)
            elif isinstance(sub_changes, list):  # sub_changes -> list
                for index, event in enumerate(sub_changes):
                    if isinstance(event, dict):
                        row = {'pid': pid,'label': str(index),
                            'mode': event.get('mode'),
                            'timestamp': event.get('timestamp')}
                        transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    df = df[['pid', 'label', 'mode', 'timestamp']]
    df.to_csv(output_file, index=False)

In [ ]:
firebase_to_csv_log_mode('/ringer_mode_events','ringer_mode_events.csv')
ringer_mode_events_data0 = pd.read_csv('/content/ringer_mode_events.csv')
ringer_mode_events_data0

,pid,label,mode,timestamp
0,pi005,1,2,1731899930595
1,pi005,2,0,1731899939692
2,pi005,3,2,1731967893903
3,pi005,4,2,1731967920328
4,pi005,5,2,1731976365082
5,pi005,-OBxFbz9VvW9fr6QSKqF,2,1731899588812
6,pi005,-OBxFeNF73wk2DQSYMv2,0,1731899598609
7,pi010,1,2,1732118658083
8,pi010,2,2,1732126050964
9,pi010,3,2,1732133177980


In [ ]:
ringer_mode_events_data0['label'] = ringer_mode_events_data0.groupby('pid').cumcount() + 1
ringer_mode_events_data0

,pid,label,mode,timestamp
0,pi005,1,2,1731899930595
1,pi005,2,0,1731899939692
2,pi005,3,2,1731967893903
3,pi005,4,2,1731967920328
4,pi005,5,2,1731976365082
5,pi005,6,2,1731899588812
6,pi005,7,0,1731899598609
7,pi010,1,2,1732118658083
8,pi010,2,2,1732126050964
9,pi010,3,2,1732133177980


### Screen events

#### Logs

In [ ]:
def firebase_to_csv_log_Screen (ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    # dictionary structure
    for pid, events in data.items():  # 'pid' = key
        pid = pid.lower()
        if isinstance(events, list):
            for index, event in enumerate(events, start=0):  #event_label_start:1
                if event is None:
                    continue
                if isinstance(event, dict):
                    row = {'pid': pid,
                        'eventLabel': index,
                        'startTime': event.get('startTime'),
                        'endTime': event.get('endTime')}
                    transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    df = df[['pid', 'eventLabel', 'startTime', 'endTime']]
    df.to_csv(output_file, index=False)

In [ ]:
firebase_to_csv_log_Screen('/screen_events','screen_events.csv')
screen_events_data0 = pd.read_csv('/content/screen_events.csv')
screen_events_data0

,pid,eventLabel,startTime,endTime
0,pi001,1,1731445600314,1731445678402
1,pi001,2,1731446736234,1731446871302
2,pi001,3,1731448956756,1731449019425
3,pi001,4,1731449052454,1731449064218
4,pi001,5,1731449228672,1731449240711
...,...,...,...,...
1383,te003,59,1731372628703,1731372630671
1384,te003,60,1731414961711,1731414975621
1385,te003,61,1731416588506,1731417319954
1386,te003,62,1731421202924,1731421203336


In [ ]:
screen_events_data = screen_events_data0.copy()
screen_events_data ["Period_sec"] = ( screen_events_data["endTime"] - screen_events_data["startTime"]) / 1000  #second
screen_events_data ["Period_min"] = ( screen_events_data["endTime"] - screen_events_data["startTime"]) / (1000 * 60)  #minute
screen_events_data ["Period_hour"] = ( screen_events_data["endTime"] - screen_events_data["startTime"]) / (1000 * 60 * 60) #hour
screen_events_data

,pid,eventLabel,startTime,endTime,Period_sec,Period_min,Period_hour
0,pi001,1,1731445600314,1731445678402,78.088,1.301467,0.021691
1,pi001,2,1731446736234,1731446871302,135.068,2.251133,0.037519
2,pi001,3,1731448956756,1731449019425,62.669,1.044483,0.017408
3,pi001,4,1731449052454,1731449064218,11.764,0.196067,0.003268
4,pi001,5,1731449228672,1731449240711,12.039,0.200650,0.003344
...,...,...,...,...,...,...,...
1383,te003,59,1731372628703,1731372630671,1.968,0.032800,0.000547
1384,te003,60,1731414961711,1731414975621,13.910,0.231833,0.003864
1385,te003,61,1731416588506,1731417319954,731.448,12.190800,0.203180
1386,te003,62,1731421202924,1731421203336,0.412,0.006867,0.000114


In [ ]:
screen_events_data['startTime_time'] = pd.to_datetime(screen_events_data['startTime'], unit='ms', utc=True).dt.tz_convert('America/New_York')
screen_events_data['endTime_time'] = pd.to_datetime(screen_events_data['endTime'], unit='ms', utc=True).dt.tz_convert('America/New_York')


In [ ]:
screen_events_data.to_csv('/content/screen_events1.csv', index=False)

In [ ]:
# Insight

screen_events_data.groupby('pid')['Period_sec'].agg(['min', 'max', 'sum', 'mean', 'std'])

,min,max,sum,mean,std
pid,,,,,
pi001,-1.575,3703.293,49617.553,171.687035,332.026607
pi002,-0.318,3140.142,41102.730,342.522750,601.329194
pi004,-371.231,5556.616,130743.668,241.670366,552.624756
te001,-0.359,74091.720,222218.870,638.559971,5071.126583
te003,-1156.804,1077.710,11010.266,186.614678,330.608934


#### Counters

In [ ]:
firebase_to_csv_counters('/screen_event_counters','screen_event_counters.csv')
screen_event_counters_data0 = pd.read_csv('/content/screen_event_counters.csv')
screen_event_counters_data0

,pid,screen_event_count
0,pi001,289
1,pi002,120
2,pi003,14
3,pi004,552
4,pi005,285
5,pi010,22
6,pi011,24
7,te001,353
8,te003,63
9,te010,64


### Setting changes

#### Counters

In [ ]:
firebase_to_csv_counters('/settings_change_counters','settings_change_counters.csv')
settings_change_counters_data0 = pd.read_csv('/content/settings_change_counters.csv')
settings_change_counters_data0

,pid,screen_event_count
0,pi003,2
1,pi004,1
2,pi005,22
3,pi010,51
4,te001,7
5,te003,2


#### Logs

In [ ]:
def firebase_to_csv_log_setting (ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    # dictionary structure
    for pid, changes in data.items():  # 'pid' = key
        if isinstance(changes, list):
            for index, change in enumerate(changes, start=0):  #event_label_start:1
                if change is None:
                    continue
                if isinstance(change, dict):
                    row = {'pid': pid,
                        'eventLabel': index,
                        'newValue' : change.get('newValue'),
                        'setting': change.get('setting'),
                        'timestamp': change.get('timestamp')}
                    transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    df = df[['pid', 'newValue', 'setting', 'timestamp']]
    df.to_csv(output_file, index=False)

In [ ]:
firebase_to_csv_log_setting('/settings_change_logs','settings_change_logs.csv')
settings_change_logs_data = pd.read_csv('/content/settings_change_logs.csv')
settings_change_logs_data

,pid,newValue,setting,timestamp
0,pi003,8,thresholdInMinute,1731896542375
1,pi003,2,vibrationFrequency,1731896576222
2,pi004,11,thresholdInMinute,1732102221921
3,te001,1,isColorEnabled,1730210574687
4,te001,0,isColorEnabled,1730210577811
5,te001,1,isColorEnabled,1730210594348
6,te001,0,isColorEnabled,1730210597325
7,te001,15,thresholdInMinute,1730210620541
8,te001,1,selectedAnimation,1730210955983
9,te001,0,selectedAnimation,1730210957747


## Working on csv files

## working on JSON data

In [ ]:
|import json
import pandas as pd
from datetime import datetime

In [ ]:
pilot_json = open('pilot_data_1.json', encoding = 'utf-8')
data_dict = json.load(pilot_json)
data_dict

In [ ]:
all_data = []
for key, values in data_dict.items():
    for value in values:
        if value is not None:
            value_id = value.copy()
            value_id['ID'] = key
            all_data.append(value_id)
df_pilot = pd.DataFrame(all_data)
df_pilot = df_pilot[['ID', 'setting', 'timestamp', 'newValue']].rename(columns={'timestamp':'Raw_timestamp'})
df_pilot

In [ ]:
df_pilot['timestamp'] = df_pilot['Raw_timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000))  #Time format
df_pilot

# Qualtrics data

## Import from Qualtrics

In [ ]:
import requests
import zipfile
import json
import time
import io

def get_qualtrics_data(api_token, survey_id):
    # API configurations
    base_url = f"https://pdx1.qualtrics.com/API/v3/surveys/{survey_id}/export-responses"
    headers = {
        "X-API-TOKEN": api_token,
        "Content-Type": "application/json"
    }

    # Start export
    export_payload = '{"format":"csv"}'
    export_response = requests.post(base_url, data=export_payload, headers=headers)
    export_progress_id = export_response.json()["result"]["progressId"]

    # Check export progress
    while True:
        progress_response = requests.get(
            f"{base_url}/{export_progress_id}",
            headers=headers
        )
        progress_status = progress_response.json()["result"]["status"]

        if progress_status == "complete":
            file_id = progress_response.json()["result"]["fileId"]
            break
        time.sleep(2)

    # Download file
    download_response = requests.get(
        f"{base_url}/{file_id}/file",
        headers=headers
    )

    # Extract zip file
    with zipfile.ZipFile(io.BytesIO(download_response.content)) as zip_file:
        return zip_file.read(zip_file.namelist()[0]).decode('utf-8')


In [ ]:
sign_up_survey = "SV_dgN8IwiCIfglbAq"
api_token = "U5xGlZmJv76LsjIXvfwB7FS9RqrqwmMb3vva3pbD"

In [ ]:
data = get_qualtrics_data(api_token, sign_up_survey)
# Then save to CSV
with open('qualtrics_data.csv', 'w') as f:
    f.write(data)

## Working on CSV file

In [ ]:
import pandas as pd

sign_up_data = pd.read_csv('/content/Sign-up survey.csv')
sign_up_data

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q13,Q13_4_TEXT,Q14,Q15,Q16,Q16_4_TEXT,Q16_8_TEXT,Q18,Q19,Q_DataPolicyViolations
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,How do you describe your gender? - Selected Ch...,How do you describe your gender? - Prefer to s...,What is the highest level of education you hav...,Choose one or more races that you consider you...,Which best describes your current employment s...,Which best describes your current employment s...,Which best describes your current employment s...,By what name would you like me to call you (th...,What is your email?,Q_DataPolicyViolations
1,"{""ImportId"":""startDate"",""timeZone"":""America/Ne...","{""ImportId"":""endDate"",""timeZone"":""America/New_...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID17""}","{""ImportId"":""QID17_4_TEXT""}","{""ImportId"":""QID20""}","{""ImportId"":""QID24""}","{""ImportId"":""QID23""}","{""ImportId"":""QID23_4_TEXT""}","{""ImportId"":""QID23_8_TEXT""}","{""ImportId"":""QID25_TEXT""}","{""ImportId"":""QID26_TEXT""}","{""ImportId"":""Q_DataPolicyViolations""}"
2,2024-10-28 17:27:50,2024-10-28 17:27:54,Survey Preview,NaN,100,4,True,2024-10-28 17:27:55,R_6KHDkdeU38MoZj3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-10-28 17:31:43,2024-10-28 17:31:47,Survey Preview,NaN,100,4,True,2024-10-28 17:31:48,R_6Kw4fLiY5gAR8ug,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-10-28 17:35:43,2024-10-28 17:35:48,Survey Preview,NaN,100,4,True,2024-10-28 17:35:48,R_6vWQH88MmG10dcB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-10-28 17:35:50,2024-10-28 17:35:55,Survey Preview,NaN,100,4,True,2024-10-28 17:35:55,R_6pJlZWMfjLdP59U,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-10-28 17:46:11,2024-10-28 17:46:26,Survey Preview,NaN,100,14,True,2024-10-28 17:46:26,R_6X0yYeEUOD73mCa,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2024-10-28 19:21:55,2024-10-28 19:22:00,Survey Preview,NaN,100,5,True,2024-10-28 19:22:01,R_61X6C9mZZ3Hzth7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2024-10-28 19:24:37,2024-10-28 19:24:41,Survey Preview,NaN,100,4,True,2024-10-28 19:24:41,R_6BqBMhKTWDVibYd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-10-28 19:24:50,2024-10-28 19:24:56,Survey Preview,NaN,100,5,True,2024-10-28 19:24:56,R_6tISpwXZKTTl1zH,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Paricipant management

## Progress check

Which day is each participant on, the state of each survey and the tutorial: unavailable, available, or completed.

### Read in the meta data

In [38]:
# path is /content/drive/MyDrive/Problematic Smartphone Usage/Ambient display/data/survey_meta.csv
survey_meta_pivoted = pd.read_csv(google_drive_path + '/survey_meta.csv')

In [39]:
import numpy as np
# Get today's date
today = pd.Timestamp.now().date()

# Convert survey1 to datetime if it's not already
survey_meta_pivoted['survey1'] = pd.to_datetime(survey_meta_pivoted['survey1']).dt.date

# Convert to numpy array to handle the date subtraction properly
survey_meta_pivoted['day_count'] = np.array([today - d for d in survey_meta_pivoted['survey1']], dtype='timedelta64[D]').astype(int) + 1
survey_meta_pivoted

,pid,host,survey1,survey2,survey3,survey4,day_count
0,pi001,NaN,2024-11-12,2024-11-15,NaN,NaN,11
1,pi002,NaN,2024-11-13,2024-11-15,NaN,NaN,10
2,pi003,NaN,2024-11-12,2024-11-15,2024-11-17,NaN,11
3,pi004,NaN,2024-11-14,2024-11-17,2024-11-20,NaN,9
4,pi005,NaN,2024-11-15,NaN,NaN,NaN,8
5,te001,NaN,2024-11-12,2024-11-12,2024-11-12,2024-11-12,11
6,te003,NaN,2024-11-11,2024-11-11,2024-11-11,NaN,12


### Read the email info and merge

In [1]:
from google.auth import default
import gspread

# Authenticate
creds, _ = default()
client = gspread.authorize(creds)

In [2]:
# Replace 'YOUR_SHEET_ID' with your actual Google Sheets ID
#https://docs.google.com/spreadsheets/d/1LBe_K-vhNXQziTF0zcpdmxZjp5XsiXF2mlVowt62rD8/edit?usp=sharing
sheet_id = "1LBe_K-vhNXQziTF0zcpdmxZjp5XsiXF2mlVowt62rD8"


In [3]:
import gspread
from google.colab import auth
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
# Authenticate Google Drive
auth.authenticate_user()

In [7]:
# Set up credentials
json_key_file = google_drive_path + "/rising-timing-385616-8bdf801b1d3b.json"  # Replace with your uploaded file's path

In [8]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_key_file, scope)

In [9]:
# Authorize client
client = gspread.authorize(credentials)

In [10]:
# Open Google Sheet by ID
sheet_id = "1LBe_K-vhNXQziTF0zcpdmxZjp5XsiXF2mlVowt62rD8"
sheet = client.open_by_key(sheet_id)


PermissionError: 

In [ ]:

# Access the first worksheet
worksheet = sheet.get_worksheet(0)

# Fetch all data
data = worksheet.get_all_records()

# Print the data
import pandas as pd
df = pd.DataFrame(data)
print(df)

## Generate rinder mail list

Generate a list of emails for certain filter (e.g., Survey2 available)

Week 1 completion

In [41]:
# filter those day_count is 7

Week 2 completion

Week 3 completion

Daily reminder

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

We need a list of participants who we are going to send the reminder emails to.